In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import sys
import random
from fake_useragent import UserAgent

In [10]:
def get_job_df(base_link,positions):
    
    df = pd.DataFrame()
    url_temp = base_link + "/jobs?q={}&start={}"
    api_url = base_link + "/viewjob?jk={}"
    ua = UserAgent()
    results_per_position=200
    for position in positions:
        
        print('----------------\nSearching for ',position.replace('+',' '))
        
        for start in range(0,results_per_position+10,10):
            user_agent = ua.random
            headers = {"User-Agent": str(user_agent)}
            url=url_temp.format(position,start)
            r=requests.get(url,headers=headers)
            soup=BeautifulSoup(r.text,"html.parser")    
            print("\r", end="")
            print("Progress: {}%: ".format(start*100//results_per_position), "▋" * (start*100//results_per_position // 2), end="")
            sys.stdout.flush()
            
            for i in soup.find_all('a',href=re.compile("/rc/clk|/company/")):
                # if tag has attribute of class
                job_link = base_link+i['href']
                time.sleep(random.randrange(3, 7, 1))
                r_req=requests.get(job_link,headers=headers)
                soup_req=BeautifulSoup(r_req.text,"html.parser") 
                try:
                    title=soup_req.find('h1',{'class': 'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'}).text
                except:
                    continue
                try:
                    location=i.find('div',{'class':'companyLocation'}).text
                except:
                    location='Unknown'
                try:
                    company=soup_req.find('div',{'class':'icl-u-lg-mr--sm icl-u-xs-mr--xs'},string=True).text
                except:
                    continue
                try:
                    desc=soup_req.find('div',{'class':'jobsearch-jobDescriptionText'}).text
                except:
                    continue
                df = df.append({"search_position":position.replace('+',' '), "job_location":location, "job_title":title, "company":company, "job_description":desc, "job_link":job_link},
                               ignore_index=True)
                
            time.sleep(random.randrange(10, 20, 1))
        print()
                
    print("===================\nWeb Scraping Complete!")
    return df

In [ ]:
ca_base_link="https://ca.indeed.com"

In [ ]:
positions=['Data+Scientist']
df_ca_ds = get_job_df(ca_base_link,positions=positions)

In [ ]:
df_ca_ds.to_csv("CA-Indeed-DS.csv")

In [ ]:
df_ca_da = get_job_df(ca_base_link,positions=['Data+Analyst'])

In [ ]:
df_ca_da.to_csv("CA-Indeed-DA.csv")

In [ ]:
df_ca_dm = get_job_df(ca_base_link,positions=['Data+Manager'])

In [ ]:
df_ca_dm

In [ ]:
df_ca_dm.to_csv("CA-Indeed-DM.csv")

In [9]:
us_base_link="https://www.indeed.com"
positions=['Data+Scientist']
df_us = get_job_df(us_base_link,positions=positions)

----------------
Searching for  Data Scientist
Progress: 100%:  ▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋
Web Scraping Complete!


In [11]:
df_us

,company,job_description,job_link,job_location,job_title,search_position
0,Flipped.ai,Data Scientists - Positions located in New Yor...,https://www.indeed.com/company/Flipped.ai/jobs...,"New York, NY",Data Scientist,Data Scientist
1,Gap Inc.,About Gap Inc.\nOur brands bridge the gaps we ...,https://www.indeed.com/rc/clk?jk=f3cd2dc1b9455...,"San Francisco, CA 94105 94105 (Financial Distr...",Associate Data Scientist,Data Scientist
2,Binare Tech,"Job Description: Design, build, test and integ...",https://www.indeed.com/company/Binare-Tech/job...,"Tampa, FL",Data Scientist,Data Scientist
3,Avenue Code,Avenue Code is the leading software consultanc...,https://www.indeed.com/rc/clk?jk=4723b2f77a1d6...,Remote,Data Scientist,Data Scientist
4,ArthurPaige,ArthurPaige is looking for a Data Scientist wi...,https://www.indeed.com/company/ArthurPaige/job...,"Fort Meade, MD",Senior Data Scientist,Data Scientist
...,...,...,...,...,...,...
125,Balyasny,Balyasny Asset Management is seeking a Junior ...,https://www.indeed.com/company/Balyasny-Asset-...,"New York, NY",Junior Data Scientist - BAM Elevate,Data Scientist
126,Neocortex Inc,We are an enterprise consulting company that p...,https://www.indeed.com/company/NeoCortex-Inc/j...,"Fremont, CA 94538 94538 (Downtown area)",SAP S/4 HANA Developer with interest in Data S...,Data Scientist
127,Phaidra,This job is 100% remote - work from anywhere i...,https://www.indeed.com/rc/clk?jk=be093e5b8d453...,"Remote in Seattle, WA 98103 98103",Data Scientist - Solutions (remote),Data Scientist
128,Data Xpander,Email at hr@dataxpander.com\nThey need to have...,https://www.indeed.com/rc/clk?jk=cdaa9e9635b32...,"Irving, TX",Data Scientist,Data Scientist


In [ ]:
df_us.to_csv("US-Indeed.csv")